In [1]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
# !mkdir /home/aistudio/external-libraries
# !pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [2]:
# 程序在/work/code目录下， 需先加入路径
import sys 
sys.path.append('/home/aistudio/work/code')
# fly_paddle是唯一需要直接调用的模块
# fly_paddle is the only module users need to interact with
import fly_paddle as fp

# args包括所有需要的参数， 贯穿于几乎所有的程序调用中
# fp.parse_args2() 根据任务初始化args, 要用到的任务包括： ‘train', 'predict'
# args is a structure containing most (if not all) parameters
# fp.parse_args2() initializes args based on the task to run, such as "train", "predict"
args, _ = fp.parse_args2('train')
print(fp.gwio.json_str(args.__dict__))

In [4]:
# 注： 根据不同的网络等等需要， args可能包含一些用不到的参数
# Attention: some parameters in args may not be used depending on the network etc.
# 两种更新args的方法： 1） args.update(**dict), 2) args.[key] = value
# Two main ways to update values in args: 1) args.update(**dict), 2) args.[key] = value
args.update(data_dir='work/data', data_name='train', residue_dbn=True, residue_extra=True)

# 网络参数 （net parameters): 
# 网络的设计主要考虑了三个RNA碱基配对的支配作用： 
#    1) 来自于全部序列的排列组合（配分）竞争，用Attention机制来模拟
#    2）来自于线性大分子的一维序列限制， 用LSTM结构来模拟
#    3）来自于局部紧邻碱基的合作（比如，一个孤立的碱基对极不稳定）， 用1D Convolution来模拟
# 所以框架由以上三个模块组成， 并在输入和输出层加了1-3个线性层。除非特意说明， 所有的隐藏层的维度为32. 
# 训练中发现高维度和深度的网络并不能给出更好的结果！
# Three main governing mechanisms for RNA base pairing are taken into consideration for the 
# design of the network architecture. 
#   1) The combinatorial configurational space of base pairing among all RNA bases, accounted for with Attention Mechanism
#   2) The quasi-1D nature of unbranched RNA polymers, accounted for with LSTM
#   3) The cooperativity of neighboring bases for stable base pairing, accounted for with 1D Convolution
# Hence the neural net comprises of three main building blocks, with addiitional linear layers for input and output. 
# The dimensions of all hidden layers are 32 unless noted otherwise
# Larger and deeper nets gave similar, but no better, performances!
args.net='seq2seq_attnlstmconv1d'  # the net name defined in paddle_nets.py
args.linear_num = 1 # the number of linear feedforward layers
args.attn_num = 1 # the number of transformer encoder layers
args.lstm_num = 1 # the number of bidirectional lstm layers
args.conv1d_num = 1 # the number of 1D convolutional layers
# 输出模块由三个线性层组成， 维度分别为32, 32, 2
# three linear layers for the final output, with dimensions of 32, 32, and 2, respectively
args.output_dim = [32, 32, 2] 
args.norm_fn = 'layer' # layer normalization
args.batch_size = 1 # 1 is used in consideration of the layer norm above
# 最后递交用的损失函数选为 softmax+bce, 也可以用 softmax+mse, 结果几乎一样
args.loss_fn = ['softmax+bce'] # softmax is needed here as the final output has a dimension of 2
args.label_tone = 'soft'
args.loss_sqrt = True # sqrt(loss) is only necessary for softmax+mse
args.loss_padding = False # exclude padded residues from loss
# 需要运行autoconfig_args()来消除参数的不一致性
# autoconfig_args() is needed to resolve inconsistencies between parameters
args = fp.autoconfig_args(args)


In [5]:
# 建立和检测模型 （Get and inspect the model）
model = fp.get_model(args)

2021-05-12 23:35:15,630 - INFO - Used net definition: /home/aistudio/work/code/paddle_nets.py
2021-05-12 23:35:15,704 - INFO - {'total_params': 36418, 'trainable_params': 36418}
2021-05-12 23:35:15,705 - INFO - Optimizer method: adam
2021-05-12 23:35:15,706 - INFO -    learning rate: 0.003
2021-05-12 23:35:15,706 - INFO -     lr_scheduler: reduced
2021-05-12 23:35:15,706 - INFO -     weight decay: none
2021-05-12 23:35:15,707 - INFO -          l1decay: 0.0001
2021-05-12 23:35:15,707 - INFO -          l2decay: 0.0001
2021-05-12 23:35:15,708 - INFO - Getting loss function: ['softmax+bce']


-------------------------------------------------------------------------------------------------------------------------------------
      Layer (type)                          Input Shape                                  Output Shape                   Param #    
   MyEmbeddingLayer-1                      [[2, 512, 10]]                                [2, 512, 10]                      0       
        Linear-1                           [[2, 512, 10]]                                [2, 512, 32]                     352      
         ReLU-1                            [[2, 512, 32]]                                [2, 512, 32]                      0       
       LayerNorm-1                         [[2, 512, 32]]                                [2, 512, 32]                     64       
        Dropout-1                          [[2, 512, 32]]                                [2, 512, 32]                      0       
     MyLinearTower-1                       [[2, 512, 10]]                 

In [6]:
# 读取数据 （read in data)
midata = fp.get_midata(args)
train_data, valid_data = fp.train_test_split(midata, test_size=0.1)

2021-05-12 23:35:15,714 - INFO - Loading data: work/data/train.pkl
2021-05-12 23:35:15,760 - INFO -    # of data: 5000,  max seqlen: 500, user seq_length: [0, 512, -1]
2021-05-12 23:35:15,761 - INFO -  residue fmt: vector, nn: 0, dbn: True, attr: False, genre: upp
2021-05-12 23:35:15,779 - INFO - Selected 5000 data sets with length range: [0, 512, -1]
2021-05-12 23:35:20,618 - INFO - Processing upp data...


In [7]:
# 训练模型， 最后的loss应该在[0.52, 0.53]区间内. 
# 每epoch需要五分钟左右(在CPU上)， 自然结束需要～20个epoch
train_loss, valid_loss = fp.train(model, train_data, num_epochs=21, validate_callback = fp.func_partial(fp.validate_in_train, midata=valid_data, save_dir='./'))

2021-05-12 23:35:21,410 - INFO - Training, data size: 4500
2021-05-12 23:35:21,508 - INFO -          batch size: 1
2021-05-12 23:35:21,509 - INFO -             shuffle: True
2021-05-12 23:35:21,510 - INFO -        # of batches: 4500
2021-05-12 23:35:21,511 - INFO -      recap interval: 151
2021-05-12 23:35:21,511 - INFO -   validate interval: 450
2021-05-12 23:35:21,512 - INFO -         # of epochs: 21
2021-05-12 23:35:21,513 - INFO -        loss padding: False
2021-05-12 23:35:22,468 - INFO - Epoch/batch: 0/   0, ibatch:    0, loss: 0.8524, std: 0.5392
2021-05-12 23:35:30,114 - INFO - loss: 0.8924, std: 0.5263
2021-05-12 23:35:36,074 - INFO - Epoch/batch: 0/ 151, ibatch:  151, loss: 0.6988, std: 0.6297
2021-05-12 23:35:41,863 - INFO - Epoch/batch: 0/ 302, ibatch:  302, loss: 0.6065, std: 0.6719
2021-05-12 23:35:57,380 - INFO - loss: 0.5724, std: 0.6974
2021-05-12 23:35:57,396 - INFO - Saved model states in: earlystop_0.5724
2021-05-12 23:35:57,397 - INFO - Saved net python code: early

Epoch 32: ReduceOnPlateau set learning rate to 0.0027.


2021-05-12 23:40:23,826 - INFO - Epoch/batch: 1/ 302, ibatch: 4802, loss: 0.5439, std: 0.6257
2021-05-12 23:40:39,182 - INFO - loss: 0.5391, std: 0.6482
2021-05-12 23:40:39,290 - INFO - Epoch/batch: 1/ 453, ibatch: 4953, loss: 0.5396, std: 0.6252
2021-05-12 23:40:44,939 - INFO - Epoch/batch: 1/ 604, ibatch: 5104, loss: 0.5315, std: 0.6126
2021-05-12 23:40:50,707 - INFO - Epoch/batch: 1/ 755, ibatch: 5255, loss: 0.5455, std: 0.6300
2021-05-12 23:41:06,312 - INFO - loss: 0.5395, std: 0.6522
2021-05-12 23:41:06,537 - INFO - Epoch/batch: 1/ 906, ibatch: 5406, loss: 0.5352, std: 0.6250
2021-05-12 23:41:12,258 - INFO - Epoch/batch: 1/1057, ibatch: 5557, loss: 0.5537, std: 0.6323
2021-05-12 23:41:18,032 - INFO - Epoch/batch: 1/1208, ibatch: 5708, loss: 0.5416, std: 0.6192
2021-05-12 23:41:34,114 - INFO - loss: 0.5382, std: 0.5862
2021-05-12 23:41:34,414 - INFO - Epoch/batch: 1/1359, ibatch: 5859, loss: 0.5469, std: 0.6269
2021-05-12 23:41:40,255 - INFO - Epoch/batch: 1/1510, ibatch: 6010, los

Epoch 53: ReduceOnPlateau set learning rate to 0.0024300000000000003.


2021-05-12 23:43:33,070 - INFO - Epoch/batch: 1/3473, ibatch: 7973, loss: 0.5384, std: 0.6229
2021-05-12 23:43:47,382 - INFO - loss: 0.5342, std: 0.6276
2021-05-12 23:43:48,310 - INFO - Epoch/batch: 1/3624, ibatch: 8124, loss: 0.5465, std: 0.6263
2021-05-12 23:43:54,213 - INFO - Epoch/batch: 1/3775, ibatch: 8275, loss: 0.5469, std: 0.6219
2021-05-12 23:44:00,371 - INFO - Epoch/batch: 1/3926, ibatch: 8426, loss: 0.5618, std: 0.6340
2021-05-12 23:44:14,705 - INFO - loss: 0.5343, std: 0.6310
2021-05-12 23:44:15,692 - INFO - Epoch/batch: 1/4077, ibatch: 8577, loss: 0.5432, std: 0.6203
2021-05-12 23:44:21,373 - INFO - Epoch/batch: 1/4228, ibatch: 8728, loss: 0.5372, std: 0.6168
2021-05-12 23:44:26,814 - INFO - Epoch/batch: 1/4379, ibatch: 8879, loss: 0.5421, std: 0.6231
2021-05-12 23:44:40,678 - INFO - loss: 0.5385, std: 0.5695
2021-05-12 23:44:41,911 - INFO - Epoch 1 average training loss: 0.5462 std: 0.6246
2021-05-12 23:44:41,920 - INFO - Epoch 1 average validate loss: 0.5379 std: 0.6168

...

2021-05-13 01:04:18,056 - INFO - Epoch/batch: 18/4379, ibatch: 85379, loss: 0.5223, std: 0.6068
2021-05-13 01:04:32,983 - INFO - loss: 0.5236, std: 0.6069
2021-05-13 01:04:32,998 - INFO - Saved model states in: earlystop_0.5236
2021-05-13 01:04:33,000 - INFO - Saved net python code: earlystop_0.5236/paddle_nets.py
2021-05-13 01:04:33,007 - INFO - Saved best model: earlystop_0.5236
2021-05-13 01:04:33,008 - INFO - Removing earlystop model: earlystop_0.5236.1
2021-05-13 01:04:34,408 - INFO - Epoch 18 average training loss: 0.5272 std: 0.6062
2021-05-13 01:04:34,413 - INFO - Epoch 18 average validate loss: 0.5237 std: 0.6049
2021-05-13 01:04:36,457 - INFO - Epoch/batch: 19/   0, ibatch: 85500, loss: 0.5341, std: 0.6134
2021-05-13 01:04:46,450 - INFO - loss: 0.5236, std: 0.6069
2021-05-13 01:04:46,522 - INFO - Saved model states in: earlystop_0.5236.1
2021-05-13 01:04:46,524 - INFO - Saved net python code: earlystop_0.5236.1/paddle_nets.py
2021-05-13 01:04:46,532 - INFO - Saved best model:

Epoch 580: ReduceOnPlateau set learning rate to 2.6183890704263157e-05.


2021-05-13 01:06:13,444 - INFO - Epoch/batch: 19/1510, ibatch: 87010, loss: 0.5104, std: 0.5897
2021-05-13 01:06:19,224 - INFO - Epoch/batch: 19/1661, ibatch: 87161, loss: 0.5301, std: 0.6105
2021-05-13 01:06:34,234 - INFO - loss: 0.5236, std: 0.6063
2021-05-13 01:06:34,689 - INFO - Epoch/batch: 19/1812, ibatch: 87312, loss: 0.5385, std: 0.6170
2021-05-13 01:06:40,363 - INFO - Epoch/batch: 19/1963, ibatch: 87463, loss: 0.5301, std: 0.6040
2021-05-13 01:06:46,042 - INFO - Epoch/batch: 19/2114, ibatch: 87614, loss: 0.5321, std: 0.6155
2021-05-13 01:07:01,290 - INFO - loss: 0.5236, std: 0.6036
2021-05-13 01:07:01,883 - INFO - Epoch/batch: 19/2265, ibatch: 87765, loss: 0.5109, std: 0.5852
2021-05-13 01:07:07,470 - INFO - Epoch/batch: 19/2416, ibatch: 87916, loss: 0.5027, std: 0.5860
2021-05-13 01:07:13,085 - INFO - Epoch/batch: 19/2567, ibatch: 88067, loss: 0.5188, std: 0.6004
2021-05-13 01:07:27,868 - INFO - loss: 0.5237, std: 0.6055
2021-05-13 01:07:28,490 - INFO - Epoch/batch: 19/2718, 

Epoch 591: ReduceOnPlateau set learning rate to 2.356550163383684e-05.


2021-05-13 01:07:53,471 - INFO - loss: 0.5237, std: 0.6023
2021-05-13 01:07:54,356 - INFO - Epoch/batch: 19/3171, ibatch: 88671, loss: 0.5362, std: 0.6142
2021-05-13 01:08:00,075 - INFO - Epoch/batch: 19/3322, ibatch: 88822, loss: 0.5251, std: 0.6098
2021-05-13 01:08:05,864 - INFO - Epoch/batch: 19/3473, ibatch: 88973, loss: 0.5223, std: 0.5976
2021-05-13 01:08:20,229 - INFO - loss: 0.5236, std: 0.6022
2021-05-13 01:08:21,287 - INFO - Epoch/batch: 19/3624, ibatch: 89124, loss: 0.5383, std: 0.6102
2021-05-13 01:08:27,443 - INFO - Epoch/batch: 19/3775, ibatch: 89275, loss: 0.5370, std: 0.6147
2021-05-13 01:08:33,471 - INFO - Epoch/batch: 19/3926, ibatch: 89426, loss: 0.5331, std: 0.6025
2021-05-13 01:08:48,143 - INFO - loss: 0.5236, std: 0.6026
2021-05-13 01:08:49,317 - INFO - Epoch/batch: 19/4077, ibatch: 89577, loss: 0.5417, std: 0.6253
2021-05-13 01:08:55,081 - INFO - Epoch/batch: 19/4228, ibatch: 89728, loss: 0.5236, std: 0.5970
2021-05-13 01:09:01,157 - INFO - Epoch/batch: 19/4379, 

In [8]:
# 读取最后一个checkpoint目录 (忽略优化器state_dict读取错误)
# read the last saved earlystop folder （ignore the error in optimizer state_dict loading)
fp.state_dict_load(model, model.validate_hist.saved_dirs[-1])
# 可以改动损失函数，检测mse损失
args.loss_fn = ['softmax+mse']
model.loss_fn = fp.get_loss_fn(args)
valid_loss = fp.validate(model, valid_data, verbose=1, batch_size=64) # try a larger batch_size, should make no difference though

2021-05-13 01:09:17,352 - INFO - Loading model states from: earlystop_0.5235
2021-05-13 01:09:17,356 - INFO - Loaded net state: earlystop_0.5235/net.state
2021-05-13 01:09:17,360 - WARNING - Error in optim state_dict loading!
2021-05-13 01:09:17,361 - INFO - Getting loss function: ['softmax+mse']
2021-05-13 01:09:17,362 - INFO - Validating, data size: 500
2021-05-13 01:09:17,362 - INFO -            batch size: 64
2021-05-13 01:09:17,363 - INFO -               shuffle: False
2021-05-13 01:09:17,363 - INFO -          # of batches: 8
2021-05-13 01:09:17,363 - INFO -        recap interval: 1
2021-05-13 01:09:17,364 - INFO -          loss padding: False
2021-05-13 01:09:20,484 - INFO - ibatch:    0, loss: 0.2294, std: 0.3232
2021-05-13 01:09:21,226 - INFO - ibatch:    1, loss: 0.2217, std: 0.3163
2021-05-13 01:09:21,963 - INFO - ibatch:    2, loss: 0.2037, std: 0.2905
2021-05-13 01:09:22,692 - INFO - ibatch:    3, loss: 0.2030, std: 0.2957
2021-05-13 01:09:23,431 - INFO - ibatch:    4, loss

In [9]:
# 读取预测数据， 存储预测结果
# 提交的结果是平均了三次运行最好checkpoint存储， 它们分别得到了0.24, 0.24, 0.242的sqrt(mse)损失， 平均后得到了0.238
# Read in prediction data, and save the predicted results
predict_data = fp.get_midata(args, data_name='predict', seq_length=-1)
y_model, std_model = fp.predict(model, predict_data, save_dir='predict.files', batch_size=1)

2021-05-13 01:09:26,581 - INFO - Loading data: work/data/predict.pkl
2021-05-13 01:09:26,583 - INFO -    # of data: 112,  max seqlen: 861, user seq_length: -1
2021-05-13 01:09:26,584 - INFO -  residue fmt: vector, nn: 0, dbn: True, attr: False, genre: upp
2021-05-13 01:09:29,946 - INFO - Predicting, data size: 112
2021-05-13 01:09:29,948 - INFO -            batch size: 1
2021-05-13 01:09:29,949 - INFO -               shuffle: False
2021-05-13 01:09:29,949 - INFO -          # of batches: 112
2021-05-13 01:09:29,950 - INFO -        recap interval: 4
2021-05-13 01:09:29,951 - INFO - Predicted files will be saved in: predict.files
100%|██████████| 112/112 [00:06<00:00, 16.26it/s]
2021-05-13 01:09:36,846 - INFO - Completed prediction of 112 samples
